In [1]:
import pandas as pd
import datetime as dt
import json
import glob
import os

In [2]:
match_id_list = ['3869685', '3869519', '3857289', '3869117', '3857278', '3857272', '3857270', '3857262']

In [52]:
pd.options.mode.chained_assignment = None  # default='warn'

for match in match_id_list:
    # Read match event files
    fname = "open-data/data/events/" + match + ".json"
    df = pd.read_json(fname)

    # Extract values from .json objects
    df['event_type'] = [event['id'] for event in df['type']]
    df['event_name'] = [event['name'] for event in df['type']]
    df['play_pattern'] = [play['name'] for play in df['play_pattern']]
    df['possession_team'] = [team['name'] for team in df['possession_team']]
    df['team'] = [team['name'] for team in df['team']]

    # Remove unnecessary columns
    df.drop(['index','type'], inplace=True, axis=1)

    team1, team2 = set(df['team'])  # Team names are added to output file names

    # Create shots dataframe
    if "Bad Behaviour" in df.event_name.values:
        cards_df = df[df['event_name']=="Bad Behaviour"]
        cards_df = cards_df[['period', 'timestamp', 'minute', 
                                'second', 'possession_team', 'play_pattern', 
                                'team', 'duration', 'bad_behaviour']]
        # Extract values form bad_behaviour objects
        cards_df['card'] = [bb['card']['name'] for bb in cards_df['bad_behaviour']]
        cards_df = cards_df.reset_index().drop(['index', 'bad_behaviour'], axis=1)

        # Create new features
        cards_df['event_type'] = 'CARD'
        cards_df['offset_in'] = dt.timedelta(seconds=1)
        cards_df['offset_out'] = dt.timedelta(seconds=5)
        cards_df['event_in_mp'] = pd.Series(dtype='datetime64[ns]')
        cards_df['event_out_mp'] = pd.Series(dtype='datetime64[ns]')

        # Save shots_df as excel file for each match
        cards_df.to_excel(f'cards/{match}_{team1}_{team2}_cards.xlsx')
    else:
        print(f'Match number {match} ({team1} vs. {team2}) had no cards registered.')
        continue

Match number 3857289 (Argentina vs. Mexico) had no cards registered.
Match number 3869117 (United States vs. Netherlands) had no cards registered.
Match number 3857272 (United States vs. England) had no cards registered.
